#1. https://python-forum.io/Thread-lambda-or-operator

In [12]:
from operator import methodcaller

In [13]:
def ask_valid1(x, conversion=lambda choice: choice.lower()):
    return conversion(x)

In [14]:
def ask_valid2(x, conversion=methodcaller('lower')):
    return conversion(x)

In [15]:
def ask_valid3(x, conversion=str.lower):
    return conversion(x)

In [16]:
%timeit ask_valid1('Name')

335 ns ± 2.85 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [17]:
%timeit ask_valid2('Name')

353 ns ± 6.49 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [18]:
%timeit ask_valid3('Name')

312 ns ± 18.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


I found some thoughts about this [here](https://treyhunner.com/2018/09/stop-writing-lambda-expressions/).

In my opinion, the third version (`str.lower`) looks more pythonic. 

`methodcaller`
If you look at the operator module sources, you find that `methodcaller` is a light-weight class. In any case, 
using it,  you implicitly include additional code to be executed. This, in turn, can slightly increase the time of execution. 

`str.lower` vs `lambda x: x.lower()`
Using `lambda` keyword is redundant in this case, `str.lower` is shorter. 

Lets look at benchmarks: 


Finally, if you have several methods, 
where the `conversion` argument should have default value (a function), 
I would suggest to define something like `_default_conversion` function, e.g. 

In [19]:
def _default_conversion(x):
    # manage all methods 
    # that include conversion function in one place.
    # Might be also useful when debugging.
    return x.lower()

#OR 

_default_conversion = str.lower
    
class A:

    def ask_valid(self, *args, conversion=_default_conversion):
        pass

    def ask_invalid(self, *args, conversion=_default_conversion):
        pass

#2. https://python-forum.io/Thread-proper-syntax-for-itertuples

You don't need to use loops for this task at all. 

Something like this: 

df.loc[df.loc[:, 'Close'] > df.loc[:, 'prev'], 'trade2'] = '+'
df.loc[df.loc[:, 'Close'] < df.loc[:, 'prev'], 'trade2'] = '-'
df.loc[df.loc[:, 'Close'] == df.loc[:, 'prev'], 'trade2'] = df.loc[((df.loc[:, 'trade2'] =='+')|df.loc[:, 'trade2'] == '-').last_valid_index()]['trade2']

should work. 



#3. https://python-forum.io/Thread-Simple-String-to-Time-within-a-pandas-dataframe

In [1]:
import pandas as pd

In [3]:
df = pd.DataFrame({'atime': ['13-06-2019 10:00', '12-06-2019 09:15'], 'x': [1,2]})

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
atime    2 non-null object
x        2 non-null int64
dtypes: int64(1), object(1)
memory usage: 112.0+ bytes


In [9]:
df.atime = pd.to_datetime(df.atime)

In [10]:
df

,atime,x
0,2019-06-13 10:00:00,1
1,2019-12-06 09:15:00,2


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
atime    2 non-null datetime64[ns]
x        2 non-null int64
dtypes: datetime64[ns](1), int64(1)
memory usage: 112.0 bytes


#4. https://python-forum.io/Thread-How-to-vstack-matrixs-in-numpy-with-different-numbers-of-columns

In [22]:
import numpy as np
a = np.array([["A1", "B1", "C1"], ["A1", "B1", "C1"]])
b = np.array([["A2", "B2"], ["A2", "B2"]])
c = np.array([["A3", "B3"], ["A3", "B3"]])
a = pd.DataFrame(a, columns=None)
b = pd.DataFrame(b, columns=None)
pd.concat([a,b]).fillna('').values

array([['A1', 'B1', 'C1'],
       ['A1', 'B1', 'C1'],
       ['A2', 'B2', ''],
       ['A2', 'B2', '']], dtype=object)

In [23]:
import pandas as pd

In [24]:
a = pd.DataFrame(a, columns=None)
b = pd.DataFrame(b, columns=None)

In [25]:
b

,0,1
0,A2,B2
1,A2,B2


In [26]:
pd.concat([a,b]).fillna('').values

array([['A1', 'B1', 'C1'],
       ['A1', 'B1', 'C1'],
       ['A2', 'B2', ''],
       ['A2', 'B2', '']], dtype=object)

In [31]:
import xarray as xr
import matplotlib.pyplot as plt
#I read the files using xarray
deficit_annual = xr.open_dataset('Rootzone_CHIRPS_era5_2000-2015_annual_SA_masked.nc')
tc = xr.open_dataset('Treecover_MODIS_2000-2015_annual_SA_masked.nc')
fig, ax = plt.subplots(figsize = (8,8))
## year I am interested in
year = 2000
i = year - 2000
# Select the indices of the low- and high-valued points
# This will results in warnings here because of NaNs;
# the NaNs should be filtered out in the indices, since they will 
# compare to False in all the comparisons, and thus not be 
# indexed by 'low' and 'high'
low = (tc[i,:,:] <= 30)    # Savanna
moderate = (tc[i,:,:] > 30) & (tc[i,:,:] < 70)    #Transitional forest
high = (tc[i,:,:] >= 70)   #Forest
# Get the coordinates for the low- and high-valued points,
# combine and transpose them to be in the correct format
y, x = np.where(low)
low_coords = np.array([x, y]).T
y, x = np.where(high)
high_coords = np.array([x, y]).T
y, x = np.where(moderate)
moderate_coords = np.array([x, y]).T
# We now calculate the distances between *all* low-valued points, and *all* high-valued points.
# This calculation scales as O^2, as does the memory cost (of the output), 
# so be wary when using it with large input sizes.
from scipy.spatial.distance import cdist, pdist
 
distances = cdist(low_coords, moderate_coords, 'euclidean')
# Now find the minimum distance along the axis of the high-valued coords, 
# which here is the second axis.
# Since we also want to find values corresponding to those minimum distances, 
# we should use the `argmin` function instead of a normal `min` function.
indices = distances.argmin(axis=1)
mindistances = distances[np.arange(distances.shape[0]), indices]
minrzs = np.array(deficit_annual[i,:,:]).flatten()[indices]
 
plt.scatter(mindistances*25, minrzs, s = 60, alpha = 0.5, color = 'goldenrod', label = 'Trasitional Forest')
 
distances = cdist(low_coords, high_coords, 'euclidean')
# Now find the minimum distance along the axis of the high-valued coords, 
# which here is the second axis.
# Since we also want to find values corresponding to those minimum distances, 
# we should use the `argmin` function instead of a normal `min` function.
indices = distances.argmin(axis=1)
mindistances = distances[np.arange(distances.shape[0]), indices]
minrzs = np.array(deficit_annual[i,:,:]).flatten()[indices]
 
plt.scatter(mindistances*25, minrzs, s = 60, alpha = 1, color = 'green', label = 'Forest')
 
 
plt.xlabel('Distance from Savanna (km)', fontsize = '14')
plt.xticks(fontsize = '14')
plt.yticks(fontsize = '14')
plt.ylabel('Rootzone storage capacity (mm/year)', fontsize = '14') 
plt.legend(fontsize = '14')
#plt.ylim((-10, 1100))   
#plt.xlim((0, 30))   

KeyError: 0